In [ ]:
import pandas as pd
from scipy.io import loadmat
import altair as alt
import numpy as np
import math

import sys,os,os.path
sys.path.append(os.path.expanduser('../src'))
                
from scipy.io import loadmat
from spinorama.load import graph_melt
from spinorama.graph import graph_freq, graph_params_default
from spinorama.compute_cea2034 import compute_cea2034

In [ ]:
def parse_graph_freq_princeton_mat(mat, suffix):
    """ Suffix can be either H or V """
    ir_name = 'IR_{:1s}'.format(suffix)
    fs_name = 'fs_{:1s}'.format(suffix)
    # compute Freq                                                                                                                   
    timestep = 1./mat[fs_name]
    print(timestep)
    # hummm                                                                                                                          
    freq = np.fft.fftfreq(2**14, d=timestep)
    # reduce spectrum to 0 to 24kHz                                                                                                  
    # lg = 2**14                                                                                                                     
    # 24k is lgs = int(lg/4)                                                                                                         
    # 20k is at 3414                                                                                                                 
    lgs = 3414
    xs = freq[0][0:lgs]
    #    
    df = pd.DataFrame({'Freq': xs})
    # loop over measurements (skipping the 5 increments)
    for i in range(0, 72, 1):
        # extract ir                                                                                                                 
        ir = mat[ir_name][i]
        # compute FFT                                                                                                                
        y = np.fft.fft(ir)
        ys = np.abs(y[0:lgs])
        # check for 0 (from manual: 0 means not measured)                                                                            
        if ys.max() == 0.0:
            print('skipping {0}deg'.format(i*5))
            continue
        # apply formula from paper to translate to dbFS                                                                              
        ys = 105.+np.log10(ys)*20.
        # interpolate to smooth response                                                                                             
        # s = InterpolatedUnivariateSpline(xs, ys)                                                                                   
        # pretty print label, per 5 deg increment, follow klippel labelling                                                          
        ilabel =i*5
        if ilabel > 180: 
            ilabel = ilabel-360
        label = '{:d}°'.format(ilabel)
        if ilabel == 0:
            label = 'On Axis'
        df[label] = ys
    # sort columns in increasing angle order 
    def a2v(angle):
        if angle == 'Freq':
            return -1000
        elif angle == 'On Axis':
            return 0
        else:
            return int(angle[:-1])

    df = df.reindex(columns=sorted(df.columns, key=lambda a: a2v(a)))
    # check empty case
    if 'On Axis' not in df.keys():
        return None
    # precision of measurement is ok above 500
    return df[df.Freq>=500]

In [ ]:
h_mat = loadmat("../datas/Princeton/Avantgarde Acoustic Solo/Avantgarde_H_IR.mat")
v_mat = loadmat("../datas/Princeton/Avantgarde Acoustic Solo/Avantgarde_V_IR.mat")

In [ ]:
df = parse_graph_freq_princeton_mat(v_mat, 'V')
df is None

In [ ]:
df = parse_graph_freq_princeton_mat(h_mat, 'H')
df is None